In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
var23 = pd.read_csv('var_2023.csv')
var24 = pd.read_csv('var_2024.csv')
var25 = pd.read_csv('var_2025_new.csv')

In [3]:
# list of all the team names, potentially helpful later

teams_list = [
    'Toronto FC', 'New York Red Bulls', 'Los Angeles FC',
       'Charlotte FC', 'FC Cincinnati', 'CF Montreal', 'New York City FC',
       'Nashville SC', 'San Jose Earthquakes', 'Columbus Crew',
       'LA Galaxy', 'DC United', 'Seattle Sounders FC', 'Atlanta United',
       'New England Revolution', 'Chicago Fire FC', 'Vancouver Whitecaps',
       'Colorado Rapids', 'Portland Timbers', 'FC Dallas', 'Austin FC',
       'Orlando City', 'Sporting Kansas City', 'Minnesota United',
       'Real Salt Lake', 'Houston Dynamo FC', 'St. Louis City',
       'Philadelphia Union'
]

### To test some of the hypotheses, we need to add a column that says which team the decision is for 

In [4]:
# This will enable us to see if the home or away team is the one getting the decision 
# all this does is check the team list, and see if it matches a team in the var decicion column
# and also checks to make sure that matches either the home or the away team 

def get_team_for(row, teams_list):
    for team in teams_list:
        if team in row['var_text']:  # check if team is mentioned in VAR text
            if team == row['home_team']:
                return row['home_team']
            elif team == row['away_team']:
                return row['away_team']
    return None  # fallback if no team match found

var23['Team_For'] = var23.apply(lambda row: get_team_for(row, teams_list), axis=1)
var24['Team_For'] = var24.apply(lambda row: get_team_for(row, teams_list), axis=1)
var25['Team_For'] = var25.apply(lambda row: get_team_for(row, teams_list), axis=1)

In [5]:
# now we can see if the team that got the decision matches the home or away team 

var23['For_Home'] = (var23['home_team']) == var23['Team_For']
var24['For_Home'] = (var24['home_team']) == var24['Team_For']
var25['For_Home'] = (var25['home_team']) == var25['Team_For']

In [6]:
# not given is the opposite of given and the double negative was confusing me 

var23['Given'] = ~var23['Not Given']
var24['Given'] = ~var24['Not Given']
var25['Given'] = ~var25['Not Given']

In [7]:
# things that are good: penalties, goals, not getting red cards
# making a new column since 'Given' is misleading, because getting a red card is not good like getting a pen is

var23["Good"] = np.where(
    ((var23["Penalty"]) & (var23["Given"])) |
    ((var23["Card"]) & (~var23["Given"])) |
    ((var23["Goal"]) & (var23["Given"])),
    True,
    False
)

var24["Good"] = np.where(
    ((var24["Penalty"]) & (var24["Given"])) |
    ((var24["Card"]) & (~var24["Given"])) |
    ((var24["Goal"]) & (var24["Given"])),
    True,
    False
)

var25["Good"] = np.where(
    ((var25["Penalty"]) & (var25["Given"])) |
    ((var25["Card"]) & (~var25["Given"])) |
    ((var25["Goal"]) & (var25["Given"])),
    True,
    False
)

### The last thing we need is the gamestate of the team that the decision is for 

In [8]:
# these compare the gamestate (winning/losing/drawing) of the team that the decision is for
# to the gamestate of the home team, after checking if the decision is for the home team or not

conditions23 = [
    # Team is winning
    ((var23["Winning"] == "Home") & (var23["For_Home"])) |
    ((var23["Winning"] == "Away") & (~var23["For_Home"])),

    # Team is tied
    (var23["Winning"] == "Tied"),

    # Team is losing
    ((var23["Winning"] == "Home") & (~var23["For_Home"])) |
    ((var23["Winning"] == "Away") & (var23["For_Home"]))
]

conditions = [
    # Team is winning
    ((var24["Winning"] == "Home") & (var24["For_Home"])) |
    ((var24["Winning"] == "Away") & (~var24["For_Home"])),

    # Team is tied
    (var24["Winning"] == "Tied"),

    # Team is losing
    ((var24["Winning"] == "Home") & (~var24["For_Home"])) |
    ((var24["Winning"] == "Away") & (var24["For_Home"]))
]

conditions25 = [
    # Team is winning
    ((var25["Winning"] == "Home") & (var25["For_Home"])) |
    ((var25["Winning"] == "Away") & (~var25["For_Home"])),

    # Team is tied
    (var25["Winning"] == "Tied"),

    # Team is losing
    ((var25["Winning"] == "Home") & (~var25["For_Home"])) |
    ((var25["Winning"] == "Away") & (var25["For_Home"]))
]

choices = ["Winning", "Tied", "Losing"]

var23["GS_For"] = np.select(conditions23, choices, default="Unknown")
var24["GS_For"] = np.select(conditions, choices, default="Unknown")
var25["GS_For"] = np.select(conditions25, choices, default="Unknown")

In [9]:
# adding a column for the year before we combine all 3 dfs

var23['Season'] = 2023
var24['Season'] = 2024
var25['Season'] = 2025

In [10]:
# combining into 1 df

var_dfs = [var23, var24, var25]
var_final = pd.concat(var_dfs, ignore_index=True)

In [11]:
# making 15 minute bins for the minute of the match so we can group multiple instances together later 
# becuase there isn't much practical difference between 74' and 77'

bins = list(range(0, 106, 15))  # 0–105 mins in 15-min bins
labels = [f"{i}-{i+14}" for i in bins[:-1]]
var_final["minute_bin"] = pd.cut(var_final["minute"].astype(int), bins=bins, labels=labels, right=False)

In [12]:
# converting the true/false columns for the types into one column that just says what even the review is for 

type_conditions = [
    var_final["Penalty"] == True,
    var_final["Card"] == True,
    var_final["Goal"] == True
]

type_choices = ["Penalty", "Card", "Goal"]

var_final["review_type"] = np.select(type_conditions, type_choices, default="Unknown")

In [15]:
# getting rid of some redundant columns 

final_cols = ['home_team', 'away_team', 'minute', 'var_text', 'home_score_at_time', 'away_score_at_time', 'referee','var_decision', 'Set Piece','Winning', 'Team_For', 'For_Home', 'Given', 'Good', 'GS_For', 'Season','minute_bin', 'review_type']
final_cols_w_prev = ['home_team', 'away_team', 'minute', 'var_text', 'home_score_at_time', 'away_score_at_time', 'referee','var_decision', 'Set Piece','Winning', 'Team_For', 'For_Home', 'Given', 'Good', 'GS_For', 'Season','minute_bin', 'review_type', 'prev1', 'prev2', 'prev3']
# var_final = var_final[final_cols]
var_final_prev = var_final[final_cols_w_prev]

In [143]:
var_final[var_final['review_type'] == 'Goal']

,home_team,away_team,minute,var_text,home_score_at_time,away_score_at_time,referee,var_decision,Set_Piece,Winning,Team_For,For_Home,Given,Good,GS_For,Season,minute_bin,review_type
0,Colorado Rapids,Sporting Kansas City,15,VAR Decision: No Goal Colorado Rapids 0-0 Spor...,0.0,0.0,Allen Chapman,VAR Decision: No Goal Colorado,False,Tied,Colorado Rapids,True,False,False,Tied,2023,15-29,Goal
1,Atlanta United,Portland Timbers,6,VAR Decision: No Goal Atlanta United 0-0 Portl...,2.0,0.0,Ted Unkel,VAR Decision: No Goal Atlanta,False,Home,Atlanta United,True,False,False,Winning,2023,0-14,Goal
2,CF Montreal,Philadelphia Union,90,VAR Decision: Goal CF Montréal 2-2 Philadelphi...,3.0,2.0,Nima Saghafi,VAR Decision: Goal CF Montréal,False,Home,Philadelphia Union,False,True,True,Losing,2023,90-104,Goal
3,CF Montreal,Philadelphia Union,90,VAR Decision: No Goal CF Montréal 2-2 Philadel...,3.0,2.0,Nima Saghafi,VAR Decision: No Goal CF,False,Home,Philadelphia Union,False,False,False,Losing,2023,90-104,Goal
4,New York City FC,DC United,19,VAR Decision: Goal New York City FC 1-0 D.C. U...,2.0,0.0,Mark Allatin,VAR Decision: Goal New York,True,Home,New York City FC,True,True,True,Winning,2023,15-29,Goal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Toronto FC,Atlanta United,23,VAR Decision: No Goal Toronto FC 0-0 Atlanta U...,0.0,0.0,Jon Freemon,VAR Decision: No Goal Toronto,False,Tied,Toronto FC,True,False,False,Tied,2025,15-29,Goal
363,San Diego FC,Toronto FC,18,VAR Decision: Goal San Diego 0-0 Toronto FC (T...,0.0,1.0,Alyssa Pennington,VAR Decision: Goal San Diego,True,Away,Toronto FC,False,True,True,Winning,2025,15-29,Goal
372,Toronto FC,Columbus Crew,81,VAR Decision: No Goal Toronto FC 1-1 Columbus ...,1.0,1.0,Chris Penso,VAR Decision: No Goal Toronto,False,Tied,Toronto FC,True,False,False,Tied,2025,75-89,Goal
374,San Diego FC,Portland Timbers,33,VAR Decision: No Goal San Diego 0-0 Portland T...,0.0,0.0,Rosendo Mendoza,VAR Decision: No Goal San,False,Tied,Portland Timbers,False,False,False,Tied,2025,30-44,Goal


In [16]:
# var_final.to_csv('/Users/arthurlennard//Desktop/MLS Data Science/var_final.csv')
# var_final_prev.to_csv('/Users/arthurlennard//Desktop/MLS Data Science/var_final_prev.csv')

### Here's a bit of exploratory data analysis

In [80]:
# does the home team get better decisions?
home_team_good = pd.crosstab(var23["For_Home"], var23["Good"])
print(home_team_good)

home_team_good = pd.crosstab(var24["For_Home"], var24["Good"])
print(home_team_good)

home_team_good = pd.crosstab(var25["For_Home"], var25["Good"])
print(home_team_good)

Good      False  True 
For_Home              
False        62     26
True         44     34
Good      False  True 
For_Home              
False        37     33
True         47     31
Good      False  True 
For_Home              
False        14     12
True         20     16


In [96]:
# this one isn't very helpful because you don't know who the decision is even for

winning_team_good = pd.crosstab(var23["Winning"], var23["Good"])
print(winning_team_good)

winning_team_good = pd.crosstab(var24["Winning"], var24["Good"])
print(winning_team_good)

winning_team_good = pd.crosstab(var25["Winning"], var25["Good"])
print(winning_team_good)

Good     False  True 
Winning              
Away        16     14
Home        39     36
Tied        51     10
Good     False  True 
Winning              
Away        21     21
Home        39     28
Tied        24     15
Good     False  True 
Winning              
Away        10      9
Home         9     12
Tied        15      7


In [82]:
# does the losing team get better decisions? 
gamestate_team_good = pd.crosstab(var23["GS_For"], var23["Good"])
print(gamestate_team_good)

gamestate_team_good = pd.crosstab(var24["GS_For"], var24["Good"])
print(gamestate_team_good)

gamestate_team_good = pd.crosstab(var25["GS_For"], var25["Good"])
print(gamestate_team_good)

Good     False  True 
GS_For               
Losing      38     11
Tied        51     10
Winning     17     39
Good     False  True 
GS_For               
Losing      24     20
Tied        24     15
Winning     36     29
Good     False  True 
GS_For               
Losing       9      5
Tied        15      7
Winning     10     16


### Basis for the final output of the project

### what percent of similar instances to mine were awarded positively?

In [18]:
# essentially filtering the data for the condiditons you select (as specific as you want)
# then finding the proportion of similar ones previously that resulted in a 'good' outcome for the team

def challenge(df, **conditions):
    """
    Calculate % of 'Good' decisions under given conditions.
    
    Example call:
        good_decision_rate(var_df_final, Penalty=True, minute_bin=4, Set_Piece=True, Winning="Home")
    """
    
    subset = df.copy()
    
    # Apply conditions one by one
    for col, val in conditions.items():
        subset = subset[subset[col] == val]
    
    if len(subset) == 0:
        return {"count": 0, "good_rate": None}  # nothing matches
    
    # Compute percentage
    good_count = subset["Good"].sum()  # since True = 1, False = 0
    total_count = len(subset)
    rate = good_count / total_count * 100
    
    return {"count": total_count, "good_count": good_count, "good_rate": rate}

In [19]:
# one minor change 

var_final = var_final.rename(columns={'Set Piece' : 'Set_Piece'})

In [45]:
result = challenge(
    var_final,
    review_type = 'Card',
    # minute_bin='75-89',
    # Set_Piece=False,
    GS_For = 'Losing',
    # Winning='Home
    # referee = 'Marcos de Oliveira'
)

print(result)

{'count': 21, 'good_count': 5, 'good_rate': 23.809523809523807}
